In [40]:
# biblioteki
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

Wykorzystany zbiór: https://sites.google.com/view/danish-fungi-dataset

In [2]:
# Ścieżki do plików CSV i folderu ze zdjęciami
metadata_train_path = '/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M-metadata/DF20M-train_metadata_PROD.csv'
metadata_test_path = '/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M-metadata/DF20M-public_test_metadata_PROD.csv'
images_folder_path = '/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M'


# Wczytanie metadanych
metadata_train = pd.read_csv(metadata_train_path)
metadata_test = pd.read_csv(metadata_test_path)

In [3]:
metadata_train.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,level2Gid,level2Name,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path
0,2862684394,2020-09-17T00:00:00,2020.0,9.0,17.0,DK,"Langesø, Morud",17215.0,Mycena crocata (Schrad.) P.Kumm.,Fungi,...,DNK.5.14_1,Nordfyns,2862684394-136762,dead wood (including bark),Uffe Nygaard,55.433810,10.200162,10.0,Unmanaged deciduous woodland,2862684394-136762.JPG
1,2238502117,2017-08-16T00:00:00,2017.0,8.0,16.0,DK,Virum,10057.0,Agaricus augustus Fr.,Fungi,...,DNK.1.28_1,Rudersdal,2238502117-320693,soil,Michael Hansen,55.800171,12.478571,5.0,garden,2238502117-320693.JPG
2,2818074328,2020-07-23T00:00:00,2020.0,7.0,23.0,DK,"Gribskov, Enghavehus",20027.0,Russula olivacea (Schaeff.) Fr.,Fungi,...,DNK.1.20_1,Hillerød,2818074328-59948,soil,Anna ploug,55.976322,12.293878,1.0,Mixed woodland (with coniferous and deciduous ...,2818074328-59948.JPG
3,2238560017,2018-09-09T00:00:00,2018.0,9.0,9.0,DK,Færgelunden,11069.0,Boletus edulis Bull.,Fungi,...,DNK.1.11_1,Frederikssund,2238560017-331396,soil,Henrik Dalgaard,55.840458,12.027228,15.0,Deciduous woodland,2238560017-331396.JPG
4,2237970493,2011-08-20T00:00:00,2011.0,8.0,20.0,DK,Tofte Skov,17278.0,Mycena pseudocorticola Kühner,Fungi,...,DNK.3.1_1,Aalborg,2237970493-5168,bark,David Boertmann,56.844840,10.192650,75.0,Forest bog,2237970493-5168.JPG


In [4]:
metadata_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3640 entries, 0 to 3639
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gbifID                3640 non-null   int64  
 1   eventDate             3639 non-null   object 
 2   year                  3639 non-null   float64
 3   month                 3639 non-null   float64
 4   day                   3639 non-null   float64
 5   countryCode           3640 non-null   object 
 6   locality              3636 non-null   object 
 7   taxonID               3640 non-null   float64
 8   scientificName        3640 non-null   object 
 9   kingdom               3640 non-null   object 
 10  phylum                3640 non-null   object 
 11  class                 3640 non-null   object 
 12  order                 3640 non-null   object 
 13  family                3640 non-null   object 
 14  genus                 3640 non-null   object 
 15  specificEpithet      

In [5]:
# Upewnij się, że kolumna 'image_path' jest w obu ramkach danych
metadata_train['full_image_path'] = metadata_train['image_path'].apply(lambda x: os.path.join(images_folder_path, x))
metadata_test['full_image_path'] = metadata_test['image_path'].apply(lambda x: os.path.join(images_folder_path, x))

In [6]:
metadata_train['full_image_path'][0]

'/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M/2862684394-136762.JPG'

In [7]:
metadata_test.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,level2Name,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,full_image_path
0,2238504270,2017-08-20T00:00:00,2017.0,8.0,20.0,DK,Store Dyrehave,20020.0,Russula nigricans (Bull.) Fr.,Fungi,...,Hillerød,2238504270-171705,soil,Dan Ingemann Jensen,55.905585,12.362367,75.0,Deciduous woodland,2238504270-171705.JPG,/Users/milenabiernacka/Desktop/studia/magister...
1,2238511676,2017-09-17T00:00:00,2017.0,9.0,17.0,DK,Store Hareskov,17238.0,Mycena haematopus (Pers.) P.Kumm.,Fungi,...,Furesø,2238511676-246956,dead wood (including bark),Tarquin Netherway,55.769675,12.401108,15.0,Deciduous woodland,2238511676-246956.JPG,/Users/milenabiernacka/Desktop/studia/magister...
2,2883271536,2020-10-18T00:00:00,2020.0,10.0,18.0,DK,Lillerød,17289.0,Mycena rosea (Schumach.) Gramberg,Fungi,...,Allerød,2883271536-291057,soil,Dan Ingemann Jensen,55.878946,12.359130,40.0,Deciduous woodland,2883271536-291057.JPG,/Users/milenabiernacka/Desktop/studia/magister...
3,2446759186,2019-10-26T00:00:00,2019.0,10.0,26.0,DK,Ollerup,10079.0,Agaricus impudicus (Rea) Pilát,Fungi,...,Svendborg,2446759186-271681,soil,Dan Schou,55.068441,10.515358,50.0,Unmanaged deciduous woodland,2446759186-271681.JPG,/Users/milenabiernacka/Desktop/studia/magister...
4,2382321410,2019-08-30T00:00:00,2019.0,8.0,30.0,DK,"Dyrehave, Valdemars Slot (inkl Pederskov)",19939.0,Russula chloroides (Krombh.) Bres.,Fungi,...,Svendborg,2382321410-191132,soil,Kirsten Bjørnsson,55.013310,10.644585,75.0,Deciduous woodland,2382321410-191132.JPG,/Users/milenabiernacka/Desktop/studia/magister...


In [8]:
metadata_train['scientificName'].value_counts()

scientificName
Mycena galericulata (Scop.) Gray       1099
Clitocybe nebularis (Batsch) Quél.     1003
Amanita muscaria (L.) Lam., 1783        863
Boletus edulis Bull.                    811
Amanita rubescens (Pers.) Gray          713
                                       ... 
Agaricus bisporus (J.E.Lange) Pilat      33
Russula laeta Jul.Schäff.                32
Clitocybe diatreta (Fr.) P.Kumm.         32
Clitocybe nitrophila Bon                 29
Russula insignis Quél.                   28
Name: count, Length: 182, dtype: int64

In [9]:
from sklearn.preprocessing import LabelEncoder

# Zakodowanie etykiet
label_encoder = LabelEncoder()
metadata_train['class_encoded'] = label_encoder.fit_transform(metadata_train['scientificName'])
metadata_test['class_encoded'] = label_encoder.transform(metadata_test['scientificName'])  # Uwaga: używamy transform, nie fit_transform

In [10]:
metadata_train.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,full_image_path,class_encoded
0,2862684394,2020-09-17T00:00:00,2020.0,9.0,17.0,DK,"Langesø, Morud",17215.0,Mycena crocata (Schrad.) P.Kumm.,Fungi,...,2862684394-136762,dead wood (including bark),Uffe Nygaard,55.433810,10.200162,10.0,Unmanaged deciduous woodland,2862684394-136762.JPG,/Users/milenabiernacka/Desktop/studia/magister...,70
1,2238502117,2017-08-16T00:00:00,2017.0,8.0,16.0,DK,Virum,10057.0,Agaricus augustus Fr.,Fungi,...,2238502117-320693,soil,Michael Hansen,55.800171,12.478571,5.0,garden,2238502117-320693.JPG,/Users/milenabiernacka/Desktop/studia/magister...,1
2,2818074328,2020-07-23T00:00:00,2020.0,7.0,23.0,DK,"Gribskov, Enghavehus",20027.0,Russula olivacea (Schaeff.) Fr.,Fungi,...,2818074328-59948,soil,Anna ploug,55.976322,12.293878,1.0,Mixed woodland (with coniferous and deciduous ...,2818074328-59948.JPG,/Users/milenabiernacka/Desktop/studia/magister...,151
3,2238560017,2018-09-09T00:00:00,2018.0,9.0,9.0,DK,Færgelunden,11069.0,Boletus edulis Bull.,Fungi,...,2238560017-331396,soil,Henrik Dalgaard,55.840458,12.027228,15.0,Deciduous woodland,2238560017-331396.JPG,/Users/milenabiernacka/Desktop/studia/magister...,40
4,2237970493,2011-08-20T00:00:00,2011.0,8.0,20.0,DK,Tofte Skov,17278.0,Mycena pseudocorticola Kühner,Fungi,...,2237970493-5168,bark,David Boertmann,56.844840,10.192650,75.0,Forest bog,2237970493-5168.JPG,/Users/milenabiernacka/Desktop/studia/magister...,89


In [11]:
# Definicja ImageDataGeneratorów
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
metadata_train.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,full_image_path,class_encoded
0,2862684394,2020-09-17T00:00:00,2020.0,9.0,17.0,DK,"Langesø, Morud",17215.0,Mycena crocata (Schrad.) P.Kumm.,Fungi,...,2862684394-136762,dead wood (including bark),Uffe Nygaard,55.433810,10.200162,10.0,Unmanaged deciduous woodland,2862684394-136762.JPG,/Users/milenabiernacka/Desktop/studia/magister...,70
1,2238502117,2017-08-16T00:00:00,2017.0,8.0,16.0,DK,Virum,10057.0,Agaricus augustus Fr.,Fungi,...,2238502117-320693,soil,Michael Hansen,55.800171,12.478571,5.0,garden,2238502117-320693.JPG,/Users/milenabiernacka/Desktop/studia/magister...,1
2,2818074328,2020-07-23T00:00:00,2020.0,7.0,23.0,DK,"Gribskov, Enghavehus",20027.0,Russula olivacea (Schaeff.) Fr.,Fungi,...,2818074328-59948,soil,Anna ploug,55.976322,12.293878,1.0,Mixed woodland (with coniferous and deciduous ...,2818074328-59948.JPG,/Users/milenabiernacka/Desktop/studia/magister...,151
3,2238560017,2018-09-09T00:00:00,2018.0,9.0,9.0,DK,Færgelunden,11069.0,Boletus edulis Bull.,Fungi,...,2238560017-331396,soil,Henrik Dalgaard,55.840458,12.027228,15.0,Deciduous woodland,2238560017-331396.JPG,/Users/milenabiernacka/Desktop/studia/magister...,40
4,2237970493,2011-08-20T00:00:00,2011.0,8.0,20.0,DK,Tofte Skov,17278.0,Mycena pseudocorticola Kühner,Fungi,...,2237970493-5168,bark,David Boertmann,56.844840,10.192650,75.0,Forest bog,2237970493-5168.JPG,/Users/milenabiernacka/Desktop/studia/magister...,89


In [15]:
metadata_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32753 entries, 0 to 32752
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gbifID                32753 non-null  int64  
 1   eventDate             32748 non-null  object 
 2   year                  32748 non-null  float64
 3   month                 32748 non-null  float64
 4   day                   32748 non-null  float64
 5   countryCode           32753 non-null  object 
 6   locality              32737 non-null  object 
 7   taxonID               32753 non-null  float64
 8   scientificName        32753 non-null  object 
 9   kingdom               32753 non-null  object 
 10  phylum                32753 non-null  object 
 11  class                 32753 non-null  object 
 12  order                 32753 non-null  object 
 13  family                32753 non-null  object 
 14  genus                 32753 non-null  object 
 15  specificEpithet    

In [16]:
# Utwórz teraz zmniejszone próbki danych
train_sample = metadata_train.sample(frac=0.2, random_state=42) 
test_sample = metadata_test.sample(frac=0.2, random_state=42) 

# Lista wszystkich unikalnych klas z oryginalnego DataFrame
all_classes = sorted(metadata_train['scientificName'].unique().tolist())

# Tworzenie generatorów z jawnie określoną listą klas
train_generator_sample = train_datagen.flow_from_dataframe(
    dataframe=train_sample,
    x_col='full_image_path',
    y_col='scientificName',
    target_size=(224, 224),
    batch_size=32,
    classes=all_classes,  # Zapewnij listę wszystkich klas
    class_mode='categorical')

test_generator_sample = test_datagen.flow_from_dataframe(
    dataframe=test_sample,
    x_col='full_image_path',
    y_col='scientificName',
    target_size=(224, 224),
    batch_size=32,
    classes=all_classes,  # Zapewnij listę wszystkich klas
    class_mode='categorical')


Found 6551 validated image filenames belonging to 182 classes.
Found 728 validated image filenames belonging to 182 classes.


In [17]:
# Wczytanie MobileNetV2 bez górnych warstw
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Dodanie górnych warstw
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)  # liczba neuronów odpowiada liczbie klas

# Zbudowanie całego modelu
model = Model(inputs=base_model.input, outputs=predictions)

# Kompilacja modelu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Trening modelu
model.fit(train_generator_sample, validation_data=test_generator_sample, epochs=10, steps_per_epoch=train_generator_sample.samples // train_generator_sample.batch_size,
          validation_steps=len(test_generator_sample))

Epoch 1/10
204/204 [==============================] - 290s 1s/step - loss: 4.3618 - accuracy: 0.1088 - val_loss: 18.8836 - val_accuracy: 0.0096
Epoch 2/10
204/204 [==============================] - 290s 1s/step - loss: 3.5540 - accuracy: 0.1902 - val_loss: 17.1339 - val_accuracy: 0.0165
Epoch 3/10
204/204 [==============================] - 295s 1s/step - loss: 3.0987 - accuracy: 0.2533 - val_loss: 16.3753 - val_accuracy: 0.0082
Epoch 4/10
204/204 [==============================] - 302s 1s/step - loss: 2.7222 - accuracy: 0.3069 - val_loss: 27.6825 - val_accuracy: 0.0124
Epoch 5/10
204/204 [==============================] - 296s 1s/step - loss: 2.4315 - accuracy: 0.3576 - val_loss: 9.5334 - val_accuracy: 0.0234
Epoch 6/10
204/204 [==============================] - 325s 2s/step - loss: 2.1972 - accuracy: 0.4094 - val_loss: 10.2197 - val_accuracy: 0.0302
Epoch 7/10
204/204 [==============================] - 354s 2s/step - loss: 1.9539 - accuracy: 0.4617 - val_loss: 10.3734 - val_accuracy: 

In [18]:
test_loss, test_accuracy = model.evaluate(test_generator_sample, steps=test_generator_sample.samples // test_generator_sample.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

22/22 [==============================] - 15s 693ms/step - loss: 11.3220 - accuracy: 0.0170
Test Loss: 11.322044372558594, Test Accuracy: 0.017045455053448677


In [19]:
model.evaluate(test_generator_sample)

23/23 [==============================] - 16s 696ms/step - loss: 11.2848 - accuracy: 0.0179


[11.284760475158691, 0.01785714365541935]

# Validation set

In [3]:
metadata_val_path = '/Users/milenabiernacka/Desktop/studia/magisterka/DF/validation/FungiCLEF2023_val_metadata_PRODUCTION.csv'
images_val_folder_path ='/Users/milenabiernacka/Desktop/studia/magisterka/DF/validation/DF21_300'

metadata_val = pd.read_csv(metadata_val_path)

In [4]:
metadata_val.head()

,observationID,month,day,countryCode,locality,taxonID,scientificName,kingdom,phylum,class,order,family,genus,specificEpithet,taxonRank,species,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,Substrate,Latitude,Longitude,CoorUncert,Habitat,image_path,filename,MetaSubstrate,class_id,poisonous
0,3008834344,1.0,1.0,DK,Kildevænget,15233,Hypoxylon fragiforme (Pers.) J.Kickx f.,Fungi,Ascomycota,Sordariomycetes,Xylariales,Hypoxylaceae,Hypoxylon,fragiforme,SPECIES,Hypoxylon fragiforme,DNK,Denmark,DNK.1_1,Hovedstaden,DNK.1.20_1,Hillerød,NaN,55.946963,12.292050,31.0,Deciduous woodland,0-3008834344.JPG,0-3008834344,wood,702,0
1,3008834345,1.0,1.0,DK,Kildevænget,14428,Gymnopilus penetrans (Fr.) Murrill,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Hymenogastraceae,Gymnopilus,penetrans,SPECIES,Gymnopilus penetrans,DNK,Denmark,DNK.1_1,Hovedstaden,DNK.1.20_1,Hillerød,NaN,55.947665,12.287160,16.0,coniferous woodland/plantation,0-3008834345.JPG,0-3008834345,wood,553,0
2,3008834345,1.0,1.0,DK,Kildevænget,14428,Gymnopilus penetrans (Fr.) Murrill,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Hymenogastraceae,Gymnopilus,penetrans,SPECIES,Gymnopilus penetrans,DNK,Denmark,DNK.1_1,Hovedstaden,DNK.1.20_1,Hillerød,NaN,55.947665,12.287160,16.0,coniferous woodland/plantation,1-3008834345.JPG,1-3008834345,wood,553,0
3,3008834353,1.0,1.0,DK,"Norup, Vammen",12226,Cordyceps militaris (L.) Link,Fungi,Ascomycota,Sordariomycetes,Hypocreales,Cordycipitaceae,Cordyceps,militaris,SPECIES,Cordyceps militaris,DNK,Denmark,DNK.2_1,Midtjylland,DNK.2.19_1,Viborg,mosses,56.529817,9.550255,30.0,coniferous woodland/plantation,0-3008834353.JPG,0-3008834353,mosses,267,0
4,3008834353,1.0,1.0,DK,"Norup, Vammen",12226,Cordyceps militaris (L.) Link,Fungi,Ascomycota,Sordariomycetes,Hypocreales,Cordycipitaceae,Cordyceps,militaris,SPECIES,Cordyceps militaris,DNK,Denmark,DNK.2_1,Midtjylland,DNK.2.19_1,Viborg,mosses,56.529817,9.550255,30.0,coniferous woodland/plantation,1-3008834353.JPG,1-3008834353,mosses,267,0


In [5]:
metadata_val.dtypes

observationID        int64
month              float64
day                float64
countryCode         object
locality            object
taxonID              int64
scientificName      object
kingdom             object
phylum              object
class               object
order               object
family              object
genus               object
specificEpithet     object
taxonRank           object
species             object
level0Gid           object
level0Name          object
level1Gid           object
level1Name          object
level2Gid           object
level2Name          object
Substrate           object
Latitude           float64
Longitude          float64
CoorUncert         float64
Habitat             object
image_path          object
filename            object
MetaSubstrate       object
class_id             int64
poisonous            int64
dtype: object

In [6]:
# Upewnij się, że kolumna 'image_path' jest w obu ramkach danych
metadata_val['full_image_path'] = metadata_val['image_path'].apply(lambda x: os.path.join(images_val_folder_path, x))

In [7]:
metadata_val

,observationID,month,day,countryCode,locality,taxonID,scientificName,kingdom,phylum,class,order,family,genus,specificEpithet,taxonRank,species,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,Substrate,Latitude,Longitude,CoorUncert,Habitat,image_path,filename,MetaSubstrate,class_id,poisonous,full_image_path
0,3008834344,1.0,1.0,DK,Kildevænget,15233,Hypoxylon fragiforme (Pers.) J.Kickx f.,Fungi,Ascomycota,Sordariomycetes,Xylariales,Hypoxylaceae,Hypoxylon,fragiforme,SPECIES,Hypoxylon fragiforme,DNK,Denmark,DNK.1_1,Hovedstaden,DNK.1.20_1,Hillerød,NaN,55.946963,12.292050,31.0,Deciduous woodland,0-3008834344.JPG,0-3008834344,wood,702,0,/Users/milenabiernacka/Desktop/studia/magister...
1,3008834345,1.0,1.0,DK,Kildevænget,14428,Gymnopilus penetrans (Fr.) Murrill,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Hymenogastraceae,Gymnopilus,penetrans,SPECIES,Gymnopilus penetrans,DNK,Denmark,DNK.1_1,Hovedstaden,DNK.1.20_1,Hillerød,NaN,55.947665,12.287160,16.0,coniferous woodland/plantation,0-3008834345.JPG,0-3008834345,wood,553,0,/Users/milenabiernacka/Desktop/studia/magister...
2,3008834345,1.0,1.0,DK,Kildevænget,14428,Gymnopilus penetrans (Fr.) Murrill,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Hymenogastraceae,Gymnopilus,penetrans,SPECIES,Gymnopilus penetrans,DNK,Denmark,DNK.1_1,Hovedstaden,DNK.1.20_1,Hillerød,NaN,55.947665,12.287160,16.0,coniferous woodland/plantation,1-3008834345.JPG,1-3008834345,wood,553,0,/Users/milenabiernacka/Desktop/studia/magister...
3,3008834353,1.0,1.0,DK,"Norup, Vammen",12226,Cordyceps militaris (L.) Link,Fungi,Ascomycota,Sordariomycetes,Hypocreales,Cordycipitaceae,Cordyceps,militaris,SPECIES,Cordyceps militaris,DNK,Denmark,DNK.2_1,Midtjylland,DNK.2.19_1,Viborg,mosses,56.529817,9.550255,30.0,coniferous woodland/plantation,0-3008834353.JPG,0-3008834353,mosses,267,0,/Users/milenabiernacka/Desktop/studia/magister...
4,3008834353,1.0,1.0,DK,"Norup, Vammen",12226,Cordyceps militaris (L.) Link,Fungi,Ascomycota,Sordariomycetes,Hypocreales,Cordycipitaceae,Cordyceps,militaris,SPECIES,Cordyceps militaris,DNK,Denmark,DNK.2_1,Midtjylland,DNK.2.19_1,Viborg,mosses,56.529817,9.550255,30.0,coniferous woodland/plantation,1-3008834353.JPG,1-3008834353,mosses,267,0,/Users/milenabiernacka/Desktop/studia/magister...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60827,3032619344,2.0,7.0,DK,Kogtved,30766,Physcia adscendens H.Olivier,Fungi,Ascomycota,Lecanoromycetes,Caliciales,Physciaceae,Physcia,adscendens,SPECIES,Physcia adscendens,DNK,Denmark,DNK.5_1,Syddanmark,DNK.5.18_1,Svendborg,bark of living trees,55.047302,10.574493,50.0,hedgerow,2-3032619344.JPG,2-3032619344,wood,-1,0,/Users/milenabiernacka/Desktop/studia/magister...
60828,3032619344,2.0,7.0,DK,Kogtved,30766,Physcia adscendens H.Olivier,Fungi,Ascomycota,Lecanoromycetes,Caliciales,Physciaceae,Physcia,adscendens,SPECIES,Physcia adscendens,DNK,Denmark,DNK.5_1,Syddanmark,DNK.5.18_1,Svendborg,bark of living trees,55.047302,10.574493,50.0,hedgerow,3-3032619344.JPG,3-3032619344,wood,-1,0,/Users/milenabiernacka/Desktop/studia/magister...
60829,3036766303,2.0,8.0,DK,Lehnskov Strand,18043,Peniophora incarnata (Pers.) P.Karst.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Peniophoraceae,Peniophora,incarnata,SPECIES,Peniophora incarnata,DNK,Denmark,DNK.5_1,Syddanmark,DNK.5.18_1,Svendborg,dead wood (including bark),55.028268,10.513541,25.0,Mixed woodland (with coniferous and deciduous ...,0-3036766303.JPG,0-3036766303,wood,1088,0,/Users/milenabiernacka/Desktop/studia/magister...
60830,3036766303,2.0,8.0,DK,Lehnskov Strand,18043,Peniophora incarnata (Pers.) P.Karst.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Peniophoraceae,Peniophora,incarnata,SPECIES,Peniophora incarnata,DNK,Denmark,DNK.5_1,Syddanmark,DNK.5.18_1,Svendborg,dead wood (including bark),55.028268,10.513541,25.0,Mixed woodland (with coniferous and deciduous ...,1-3036766303.JPG,1-3036766303,wood,1088,0,/Users/milenabi

In [8]:
metadata_val['full_image_path'].values

array(['/Users/milenabiernacka/Desktop/studia/magisterka/DF/validation/DF21_300/0-3008834344.JPG',
       '/Users/milenabiernacka/Desktop/studia/magisterka/DF/validation/DF21_300/0-3008834345.JPG',
       '/Users/milenabiernacka/Desktop/studia/magisterka/DF/validation/DF21_300/1-3008834345.JPG',
       ...,
       '/Users/milenabiernacka/Desktop/studia/magisterka/DF/validation/DF21_300/0-3036766303.JPG',
       '/Users/milenabiernacka/Desktop/studia/magisterka/DF/validation/DF21_300/1-3036766303.JPG',
       '/Users/milenabiernacka/Desktop/studia/magisterka/DF/validation/DF21_300/0-3036766304.JPG'],
      dtype=object)

In [9]:
metadata_val['class_id'].value_counts().sort_index()

class_id
-1       21954
 0          44
 2           9
 3          10
 4          97
         ...  
 1595      194
 1596      131
 1598       79
 1599        2
 1602       68
Name: count, Length: 1085, dtype: int64

In [10]:
# Lets first handle numerical features with nan value
numerical_nan = [feature for feature in metadata_val.columns if metadata_val[feature].isna().sum()>1 and metadata_val[feature].dtypes!='O']
numerical_nan

['CoorUncert']

In [11]:
metadata_val.columns

Index(['observationID', 'month', 'day', 'countryCode', 'locality', 'taxonID',
       'scientificName', 'kingdom', 'phylum', 'class', 'order', 'family',
       'genus', 'specificEpithet', 'taxonRank', 'species', 'level0Gid',
       'level0Name', 'level1Gid', 'level1Name', 'level2Gid', 'level2Name',
       'Substrate', 'Latitude', 'Longitude', 'CoorUncert', 'Habitat',
       'image_path', 'filename', 'MetaSubstrate', 'class_id', 'poisonous',
       'full_image_path'],
      dtype='object')

In [12]:
metadata_val.isnull().sum()

observationID        0
month                0
day                  0
countryCode          0
locality            19
taxonID              0
scientificName       0
kingdom              0
phylum               0
class               45
order              168
family             438
genus                0
specificEpithet      0
taxonRank            0
species              0
level0Gid          193
level0Name         193
level1Gid          193
level1Name         193
level2Gid          194
level2Name         194
Substrate          328
Latitude             0
Longitude            0
CoorUncert          11
Habitat             15
image_path           0
filename             0
MetaSubstrate        8
class_id             0
poisonous            0
full_image_path      0
dtype: int64

In [13]:
print(metadata_val['species'].value_counts())

species
Micarea lignaria                458
Trametes versicolor             457
Fomitopsis pinicola             355
Hypholoma fasciculare           318
Stereum hirsutum                307
                               ... 
Tricholoma sudum                  1
Inocybe obscurobadia              1
Cortinarius xantho-ochraceus      1
Arthonia varians                  1
Guepinia helvelloides             1
Name: count, Length: 2673, dtype: int64


In [14]:
# Sprawdzenie unikalnych wartości w kolumnach
print(metadata_val['species'].nunique())
print(metadata_val['poisonous'].value_counts())

2673
poisonous
0    58300
1     2532
Name: count, dtype: int64


In [15]:
metadata = metadata_val[['species', 'poisonous', 'image_path', 'filename', 'full_image_path']]

In [16]:
metadata

,species,poisonous,image_path,filename,full_image_path
0,Hypoxylon fragiforme,0,0-3008834344.JPG,0-3008834344,/Users/milenabiernacka/Desktop/studia/magister...
1,Gymnopilus penetrans,0,0-3008834345.JPG,0-3008834345,/Users/milenabiernacka/Desktop/studia/magister...
2,Gymnopilus penetrans,0,1-3008834345.JPG,1-3008834345,/Users/milenabiernacka/Desktop/studia/magister...
3,Cordyceps militaris,0,0-3008834353.JPG,0-3008834353,/Users/milenabiernacka/Desktop/studia/magister...
4,Cordyceps militaris,0,1-3008834353.JPG,1-3008834353,/Users/milenabiernacka/Desktop/studia/magister...
...,...,...,...,...,...
60827,Physcia adscendens,0,2-3032619344.JPG,2-3032619344,/Users/milenabiernacka/Desktop/studia/magister...
60828,Physcia adscendens,0,3-3032619344.JPG,3-3032619344,/Users/milenabiernacka/Desktop/studia/magister...
60829,Peniophora incarnata,0,0-3036766303.JPG,0-3036766303,/Users/milenabiernacka/Desktop/studia/magister...
60830,Peniophora incarnata,0,1-3036766303.JPG,1-3036766303,/Users/milenabiernacka/Desktop/studia/magister...


In [17]:
label_encoder = LabelEncoder()
metadata['species'] = label_encoder.fit_transform(metadata['species'])
print(metadata.head(20))

    species  poisonous        image_path      filename  \
0      1227          0  0-3008834344.JPG  0-3008834344   
1      1023          0  0-3008834345.JPG  0-3008834345   
2      1023          0  1-3008834345.JPG  1-3008834345   
3       471          0  0-3008834353.JPG  0-3008834353   
4       471          0  1-3008834353.JPG  1-3008834353   
5      2399          0  0-3008834354.JPG  0-3008834354   
6      2646          0  0-3008834362.JPG  0-3008834362   
7      1442          0  0-3008834364.JPG  0-3008834364   
8      1442          0  1-3008834364.JPG  1-3008834364   
9      1442          0  2-3008834364.JPG  2-3008834364   
10     2402          0  0-3008834366.JPG  0-3008834366   
11     2029          0  0-3008834372.JPG  0-3008834372   
12     2029          0  1-3008834372.JPG  1-3008834372   
13     1808          0  0-3008834375.JPG  0-3008834375   
14     1645          0  0-3008834376.JPG  0-3008834376   
15     1645          0  1-3008834376.JPG  1-3008834376   
16     1645   

/var/folders/s0/p8ldq2yj3j71tzmvgws_cyp40000gn/T/ipykernel_1354/4021330278.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata['species'] = label_encoder.fit_transform(metadata['species'])


In [18]:
from sklearn.model_selection import train_test_split

# Podziel dane na treningowe i testowe
train_df, test_df = train_test_split(metadata, test_size=0.2, random_state=42)

In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48665 entries, 43851 to 56422
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   species          48665 non-null  int64 
 1   poisonous        48665 non-null  int64 
 2   image_path       48665 non-null  object
 3   filename         48665 non-null  object
 4   full_image_path  48665 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.2+ MB


In [20]:
# Konwersja zakodowanych etykiet na ciągi znaków, jeśli chcesz użyć class_mode='sparse'
train_df['species_str'] = train_df['species'].astype(str)
# Konwersja zakodowanych etykiet na ciągi znaków, jeśli chcesz użyć class_mode='sparse'
test_df['species_str'] = test_df['species'].astype(str)

In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48665 entries, 43851 to 56422
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   species          48665 non-null  int64 
 1   poisonous        48665 non-null  int64 
 2   image_path       48665 non-null  object
 3   filename         48665 non-null  object
 4   full_image_path  48665 non-null  object
 5   species_str      48665 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.6+ MB


In [22]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12167 entries, 48791 to 15460
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   species          12167 non-null  int64 
 1   poisonous        12167 non-null  int64 
 2   image_path       12167 non-null  object
 3   filename         12167 non-null  object
 4   full_image_path  12167 non-null  object
 5   species_str      12167 non-null  object
dtypes: int64(2), object(4)
memory usage: 665.4+ KB


In [23]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42) # 20% danych na walidację

In [31]:
# Stwórz generator obrazów
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Enhanced ImageDataGenerator with data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Stwórz generatory danych treningowych i testowych
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=images_val_folder_path,
    x_col='full_image_path', # Nazwa kolumny z pełnymi ścieżkami do obrazów
    y_col='species_str', # Nazwy kolumn, które chcesz użyć jako etykiety
    class_mode='sparse', # Dla wielu etykiet
    target_size=(224, 224), # Zmiana rozmiaru obrazów
    batch_size=32,
    shuffle=True # Mieszanie danych
)

test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=images_val_folder_path,
    x_col='full_image_path',
    y_col='species_str',
    class_mode='sparse',
    target_size=(224, 224),
    batch_size=32,
    shuffle=False # Bez mieszania, aby móc porównać wyniki
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=images_val_folder_path,
    x_col='full_image_path',
    y_col='species_str',
    class_mode='sparse',
    target_size=(224, 224),
    batch_size=32,
    shuffle=False # Bez mieszania, aby móc porównać wyniki
)

Found 38932 validated image filenames belonging to 2557 classes.
Found 12167 validated image filenames belonging to 1975 classes.
Found 9733 validated image filenames belonging to 1818 classes.


In [33]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Assuming `train_df['species_str']` contains your labels
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes)

class_weight_dict = dict(enumerate(class_weights))


In [35]:
from tensorflow.keras.optimizers.legacy import Adam


# Wczytanie MobileNetV2 bez górnych warstw
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Continue from your base model setup
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout
predictions = Dense(len(np.unique(train_generator.classes)), activation='softmax')(x)


# Zbudowanie całego modelu
model = Model(inputs=base_model.input, outputs=predictions)

# Kompilacja modelu
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Trening modelu
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    class_weight=class_weight_dict  # Include class weights here
)

Epoch 1/20
1216/1216 [==============================] - 2285s 2s/step - loss: 7.9275 - accuracy: 6.6838e-04 - val_loss: 7.8707 - val_accuracy: 0.0028
Epoch 2/20
1216/1216 [==============================] - 3613s 3s/step - loss: 7.8778 - accuracy: 1.5424e-04 - val_loss: 7.8680 - val_accuracy: 3.0839e-04
Epoch 3/20
1216/1216 [==============================] - 2662s 2s/step - loss: 7.8713 - accuracy: 5.1414e-05 - val_loss: 7.8662 - val_accuracy: 2.0559e-04
Epoch 4/20
1216/1216 [==============================] - 2911s 2s/step - loss: 7.8733 - accuracy: 2.5707e-05 - val_loss: 7.8642 - val_accuracy: 2.0559e-04
Epoch 5/20
1216/1216 [==============================] - 2122s 2s/step - loss: 7.8686 - accuracy: 0.0000e+00 - val_loss: 7.8626 - val_accuracy: 2.0559e-04
Epoch 6/20
1216/1216 [==============================] - 2182s 2s/step - loss: 7.8694 - accuracy: 2.5707e-05 - val_loss: 7.8610 - val_accuracy: 5.1398e-04
Epoch 7/20
1216/1216 [==============================] - 2199s 2s/step - loss: 7.

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_generator)

381/381 [==============================] - 84s 221ms/step - loss: 28.4713 - accuracy: 4.9314e-04


[28.471328735351562, 0.0004931371659040451]

In [ ]:
model_path = '/Users/milenabiernacka/Desktop/studia/magisterka/Magisterka/trained_models/model2/'

In [ ]:
model.save(model_path)

INFO:tensorflow:Assets written to: /Users/milenabiernacka/Desktop/studia/magisterka/Magisterka/trained_models/assets


INFO:tensorflow:Assets written to: /Users/milenabiernacka/Desktop/studia/magisterka/Magisterka/trained_models/assets


In [1]:
print('a')

a
